<a href="https://colab.research.google.com/github/namoshi/colab/blob/master/light_gbm_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import lightgbm as lgb

# データセットをロードする
data = load_breast_cancer()

# シード
seed = 42

# 説明変数をXに、目的変数をyに格納する
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)


In [2]:
# Datasetオブジェクトの作成 (異なるポイント①)
d_train = lgb.Dataset(data=X_train,label=y_train)
d_test = lgb.Dataset(data=X_test,label=y_test)

# パラメータ設定
params = {
    'objective':'binary', # 二値分類タスク(回帰であればregression)
    'metric':'binary_logloss', # objectiveを定義しているので自動入力されるが、明示しておく
    'n_estimators':10000, # early_stopping使用時は大きな値
    'verbosity': -1, # コマンドライン出力しない設定
    'random_state':seed
}

# モデルの構築・学習 (異なるポイント②)
gbm = lgb.train(params, d_train, valid_sets=[d_test], # early_stoppingの評価用データ
                callbacks=[lgb.early_stopping(stopping_rounds=10, verbose=True)] # early_stopping用コールバック関数
               )

# テストデータで予測
y_pred = gbm.predict(X_test)
y_pred = y_pred.round(0) # 丸め込み

# テストデータの評価
accuracy = accuracy_score(y_pred,y_test)
print(accuracy)
# 0.9736842105263158


/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[45]	valid_0's binary_logloss: 0.10416
0.9736842105263158


In [3]:
# パラメータ設定
params = {
    'objective':'binary',
    'metric':'binary_logloss',
    'n_estimators':10000,
    'verbosity': -1,
    'random_state':seed
}

# インスタンスの作成 (異なるポイント①)
gbm = lgb.LGBMClassifier(**params)  # 回帰であればlgb.LGBMRegressor()

# モデルの学習 (異なるポイント②)
gbm.fit(X_train, y_train, eval_metric='bainry_logloss', eval_set=[(X_test, y_test)],
        callbacks=[lgb.early_stopping(stopping_rounds=10, verbose=True)]
       )

# テストデータで予測
y_pred = gbm.predict(X_test)
y_pred = y_pred.round(0) # 丸め込み

# テストデータの評価
accuracy = accuracy_score(y_pred,y_test)
print(accuracy)
# 0.9736842105263158


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[45]	valid_0's binary_logloss: 0.10416
0.9736842105263158
